<a href="https://colab.research.google.com/github/Bittu-kumar2003/A_Glimpse_Life_App/blob/master/Image_model_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**LSTM_Model**

In [ ]:
import os
import numpy as np
from datetime import datetime
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Reshape, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

def load_tif_image(file_path, target_size=(128, 128)):
    img = load_img(file_path, color_mode='grayscale', target_size=target_size)
    img = img_to_array(img)
    img = img / 255.0  # Normalize to [0, 1]
    return img

def load_dataset_from_directory(directory, target_size=(128, 128)):
    images = []
    dates = []
    stations = []

    for file_name in sorted(os.listdir(directory)):
        if file_name.endswith('.tif'):
            try:
                station = file_name[:-15]
                date_str = file_name[-14:-4]
                date = datetime.strptime(date_str, '%Y-%m-%d')

                img_path = os.path.join(directory, file_name)
                print(f"Loading image: {img_path}")
                img = load_tif_image(img_path, target_size=target_size)
                images.append(img)
                dates.append(date)
                stations.append(station)
            except ValueError as e:
                print(f"Skipping file {file_name}: {e}")

    return np.array(images), np.array(dates), np.array(stations)

def prepare_data(images, sequence_length=10, prediction_offsets=[1, 3, 5]):
    X = []
    y_targets = [[] for _ in prediction_offsets]
    for i in range(len(images) - sequence_length - max(prediction_offsets) + 1):
        X.append(images[i:i + sequence_length])
        for j, offset in enumerate(prediction_offsets):
            y_targets[j].append(images[i + sequence_length + offset - 1])
    return np.array(X), [np.array(y) for y in y_targets]

def create_lstm_model(sequence_length, image_shape, num_predictions):
    model_input = Input(shape=(sequence_length, *image_shape))

    x = Reshape((sequence_length, np.prod(image_shape)))(model_input)

    x = LSTM(128, return_sequences=True)(x)
    x = LSTM(128)(x)

    outputs = []
    for _ in range(num_predictions):
        x_out = Dense(np.prod(image_shape), activation='sigmoid')(x)
        output = Reshape(image_shape)(x_out)
        outputs.append(output)

    model = Model(inputs=model_input, outputs=outputs)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

    return model

def predict_next_images(model, input_sequence, num_predictions):
    input_sequence = np.expand_dims(input_sequence, axis=0)
    predicted_images = model.predict(input_sequence)
    return [predicted.squeeze() for predicted in predicted_images]

# Example usage
directory = '/content/drive/MyDrive/Dataset_img/one_month_NO2_2019'
stations = ['Delhi', 'Mumbai', 'Kolkata', 'Jaipur', 'Bengaluru', 'Bhopal', 'Chennai']
target_size = (128, 128)
sequence_length = 10
prediction_offsets = [1, 3, 5]

all_images, all_dates, all_stations = load_dataset_from_directory(directory, target_size=target_size)

X, y_targets = prepare_data(all_images, sequence_length=sequence_length, prediction_offsets=prediction_offsets)

image_shape = (128, 128, 1)
num_predictions = len(prediction_offsets)
model = create_lstm_model(sequence_length, image_shape, num_predictions)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X, y_targets, epochs=12, batch_size=4, validation_split=0.1, callbacks=[early_stopping])

unique_stations = np.unique(all_stations)
overall_y_true = [[] for _ in prediction_offsets]
overall_y_pred = [[] for _ in prediction_offsets]

for station in unique_stations:
    station_indices = np.where(all_stations == station)[0]
    if len(station_indices) > sequence_length + max(prediction_offsets) - 1:
        index = station_indices[0]
        input_sequence = X[index]
        y_true_list = [y[index] for y in y_targets]
        predicted_images = predict_next_images(model, input_sequence, num_predictions)

        for i, (y_true, predicted) in enumerate(zip(y_true_list, predicted_images)):
            overall_y_true[i].append(y_true)
            overall_y_pred[i].append(predicted)

        for i, prediction in enumerate(predicted_images):
            plt.imshow(prediction.squeeze(), cmap='gray')
            plt.title(f'Predicted Image {prediction_offsets[i]} Days Ahead for {station}')
            plt.show()

for i, offset in enumerate(prediction_offsets):
    overall_y_true_np = np.array(overall_y_true[i])
    overall_y_pred_np = np.array(overall_y_pred[i])
    overall_r2 = r2_score(overall_y_true_np.reshape(len(overall_y_true_np), -1), overall_y_pred_np.reshape(len(overall_y_pred_np), -1))
    overall_mse = mean_squared_error(overall_y_true_np.reshape(len(overall_y_true_np), -1), overall_y_pred_np.reshape(len(overall_y_pred_np), -1))
    overall_mae = mean_absolute_error(overall_y_true_np.reshape(len(overall_y_true_np), -1), overall_y_pred_np.reshape(len(overall_y_pred_np), -1))
    print(f"Overall R² score for {offset} days ahead: {overall_r2:.4f}")
    print(f"Overall MSE for {offset} days ahead: {overall_mse:.4f}")
    print(f"Overall MAE for {offset} days ahead: {overall_mae:.4f}")


**CNN_Model**

In [ ]:
import os
import numpy as np
from datetime import datetime
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Reshape, Input, TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

def load_tif_image(file_path, target_size=(128, 128)):
    img = load_img(file_path, color_mode='grayscale', target_size=target_size)
    img = img_to_array(img)
    img = img / 255.0  # Normalize to [0, 1]
    return img

def load_dataset_from_directory(directory, target_size=(128, 128)):
    images = []
    dates = []
    stations = []

    for file_name in sorted(os.listdir(directory)):
        if file_name.endswith('.tif'):
            try:
                station = file_name[:-15]
                date_str = file_name[-14:-4]
                date = datetime.strptime(date_str, '%Y-%m-%d')

                img_path = os.path.join(directory, file_name)
                print(f"Loading image: {img_path}")
                img = load_tif_image(img_path, target_size=target_size)
                images.append(img)
                dates.append(date)
                stations.append(station)
            except ValueError as e:
                print(f"Skipping file {file_name}: {e}")

    return np.array(images), np.array(dates), np.array(stations)

def prepare_data(images, sequence_length=10, prediction_offsets=[1, 3, 5]):
    X = []
    y_targets = [[] for _ in prediction_offsets]
    for i in range(len(images) - sequence_length - max(prediction_offsets) + 1):
        X.append(images[i:i + sequence_length])
        for j, offset in enumerate(prediction_offsets):
            y_targets[j].append(images[i + sequence_length + offset - 1])
    return np.array(X), [np.array(y) for y in y_targets]

def create_cnn_model(sequence_length, image_shape, num_predictions):
    model_input = Input(shape=(sequence_length, *image_shape))

    x = TimeDistributed(Conv2D(16, (3, 3), activation='relu'))(model_input)
    x = TimeDistributed(MaxPooling2D((2, 2)))(x)
    x = TimeDistributed(Conv2D(32, (3, 3), activation='relu'))(x)
    x = TimeDistributed(MaxPooling2D((2, 2)))(x)
    x = TimeDistributed(Flatten())(x)
    x = TimeDistributed(Dense(128, activation='relu'))(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)

    outputs = []
    for _ in range(num_predictions):
        x_out = Dense(np.prod(image_shape), activation='sigmoid')(x)
        output = Reshape(image_shape)(x_out)
        outputs.append(output)

    model = Model(inputs=model_input, outputs=outputs)
    model.compile(optimizer='adam', loss='mean_squared_error')

    return model

def predict_next_images(model, input_sequence, num_predictions):
    input_sequence = np.expand_dims(input_sequence, axis=0)
    predicted_images = model.predict(input_sequence)
    return [predicted.squeeze() for predicted in predicted_images]

# Example usage
directory = '/content/drive/MyDrive/Dataset_img/one_month_NO2_2019'
stations = ['Delhi', 'Mumbai', 'Kolkata', 'Jaipur', 'Bengaluru', 'Bhopal', 'Chennai']
target_size = (128, 128)
sequence_length = 10
prediction_offsets = [1, 3, 5]

all_images, all_dates, all_stations = load_dataset_from_directory(directory, target_size=target_size)

X, y_targets = prepare_data(all_images, sequence_length=sequence_length, prediction_offsets=prediction_offsets)

image_shape = (128, 128, 1)
num_predictions = len(prediction_offsets)
model = create_cnn_model(sequence_length, image_shape, num_predictions)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X, y_targets, epochs=12, batch_size=4, validation_split=0.1, callbacks=[early_stopping])

unique_stations = np.unique(all_stations)
overall_y_true = [[] for _ in prediction_offsets]
overall_y_pred = [[] for _ in prediction_offsets]

for station in unique_stations:
    station_indices = np.where(all_stations == station)[0]
    if len(station_indices) > sequence_length + max(prediction_offsets) - 1:
        index = station_indices[0]
        input_sequence = X[index]
        y_true_list = [y[index] for y in y_targets]
        predicted_images = predict_next_images(model, input_sequence, num_predictions)

        for i, (y_true, predicted) in enumerate(zip(y_true_list, predicted_images)):
            overall_y_true[i].append(y_true)
            overall_y_pred[i].append(predicted)

        for i, prediction in enumerate(predicted_images):
            plt.imshow(prediction.squeeze(), cmap='gray')
            plt.title(f'Predicted Image {prediction_offsets[i]} Days Ahead for {station}')
            plt.show()

for i, offset in enumerate(prediction_offsets):
    overall_y_true_np = np.array(overall_y_true[i])
    overall_y_pred_np = np.array(overall_y_pred[i])
    overall_r2 = r2_score(overall_y_true_np.reshape(len(overall_y_true_np), -1), overall_y_pred_np.reshape(len(overall_y_pred_np), -1))
    overall_mse = mean_squared_error(overall_y_true_np.reshape(len(overall_y_true_np), -1), overall_y_pred_np.reshape(len(overall_y_pred_np), -1))
    overall_mae = mean_absolute_error(overall_y_true_np.reshape(len(overall_y_true_np), -1), overall_y_pred_np.reshape(len(overall_y_pred_np), -1))
    print(f"Overall R² score for {offset} days ahead: {overall_r2:.4f}")
    print(f"Overall MSE for {offset} days ahead: {overall_mse:.4f}")
    print(f"Overall MAE for {offset} days ahead: {overall_mae:.4f}")


**CNN+LSTM**

In [ ]:
import os
import numpy as np
from datetime import datetime
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Reshape, Input, TimeDistributed, LSTM
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

def load_tif_image(file_path, target_size=(128, 128)):
    img = load_img(file_path, color_mode='grayscale', target_size=target_size)
    img = img_to_array(img)
    img = img / 255.0  # Normalize to [0, 1]
    return img

def load_dataset_from_directory(directory, target_size=(128, 128)):
    images = []
    dates = []
    stations = []

    for file_name in sorted(os.listdir(directory)):
        if file_name.endswith('.tif'):
            try:
                station = file_name[:-15]
                date_str = file_name[-14:-4]
                date = datetime.strptime(date_str, '%Y-%m-%d')

                img_path = os.path.join(directory, file_name)
                print(f"Loading image: {img_path}")
                img = load_tif_image(img_path, target_size=target_size)
                images.append(img)
                dates.append(date)
                stations.append(station)
            except ValueError as e:
                print(f"Skipping file {file_name}: {e}")

    return np.array(images), np.array(dates), np.array(stations)

def prepare_data(images, sequence_length=10, prediction_offsets=[1, 3, 5]):
    X = []
    y_targets = [[] for _ in prediction_offsets]
    for i in range(len(images) - sequence_length - max(prediction_offsets) + 1):
        X.append(images[i:i + sequence_length])
        for j, offset in enumerate(prediction_offsets):
            y_targets[j].append(images[i + sequence_length + offset - 1])
    return np.array(X), [np.array(y) for y in y_targets]

def create_cnn_lstm_model(sequence_length, image_shape, num_predictions):
    model_input = Input(shape=(sequence_length, *image_shape))

    x = TimeDistributed(Conv2D(16, (3, 3), activation='relu'))(model_input)
    x = TimeDistributed(MaxPooling2D((2, 2)))(x)
    x = TimeDistributed(Conv2D(32, (3, 3), activation='relu'))(x)
    x = TimeDistributed(MaxPooling2D((2, 2)))(x)
    x = TimeDistributed(Flatten())(x)

    x = LSTM(128, return_sequences=True)(x)
    x = LSTM(128)(x)

    outputs = []
    for _ in range(num_predictions):
        x_out = Dense(np.prod(image_shape), activation='sigmoid')(x)
        output = Reshape(image_shape)(x_out)
        outputs.append(output)

    model = Model(inputs=model_input, outputs=outputs)
    model.compile(optimizer='adam', loss='mean_squared_error')

    return model

def predict_next_images(model, input_sequence, num_predictions):
    input_sequence = np.expand_dims(input_sequence, axis=0)
    predicted_images = model.predict(input_sequence)
    return [predicted.squeeze() for predicted in predicted_images]

# Example usage
directory = '/content/drive/MyDrive/Dataset_img/one_month_NO2_2019'
stations = ['Delhi', 'Mumbai', 'Kolkata', 'Jaipur', 'Bengaluru', 'Bhopal', 'Chennai']
target_size = (128, 128)
sequence_length = 10
prediction_offsets = [1, 3, 5]

all_images, all_dates, all_stations = load_dataset_from_directory(directory, target_size=target_size)

X, y_targets = prepare_data(all_images, sequence_length=sequence_length, prediction_offsets=prediction_offsets)

image_shape = (128, 128, 1)
num_predictions = len(prediction_offsets)
model = create_cnn_lstm_model(sequence_length, image_shape, num_predictions)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X, y_targets, epochs=12, batch_size=4, validation_split=0.1, callbacks=[early_stopping])

unique_stations = np.unique(all_stations)
overall_y_true = [[] for _ in prediction_offsets]
overall_y_pred = [[] for _ in prediction_offsets]

for station in unique_stations:
    station_indices = np.where(all_stations == station)[0]
    if len(station_indices) > sequence_length + max(prediction_offsets) - 1:
        index = station_indices[0]
        input_sequence = X[index]
        y_true_list = [y[index] for y in y_targets]
        predicted_images = predict_next_images(model, input_sequence, num_predictions)

        for i, (y_true, predicted) in enumerate(zip(y_true_list, predicted_images)):
            overall_y_true[i].append(y_true)
            overall_y_pred[i].append(predicted)

        for i, prediction in enumerate(predicted_images):
            plt.imshow(prediction.squeeze(), cmap='gray')
            plt.title(f'Predicted Image {prediction_offsets[i]} Days Ahead for {station}')
            plt.show()

for i, offset in enumerate(prediction_offsets):
    overall_y_true_np = np.array(overall_y_true[i])
    overall_y_pred_np = np.array(overall_y_pred[i])
    overall_r2 = r2_score(overall_y_true_np.reshape(len(overall_y_true_np), -1), overall_y_pred_np.reshape(len(overall_y_pred_np), -1))
    overall_mse = mean_squared_error(overall_y_true_np.reshape(len(overall_y_true_np), -1), overall_y_pred_np.reshape(len(overall_y_pred_np), -1))
    overall_mae = mean_absolute_error(overall_y_true_np.reshape(len(overall_y_true_np), -1), overall_y_pred_np.reshape(len(overall_y_pred_np), -1))
    print(f"Overall R² score for {offset} days ahead: {overall_r2:.4f}")
    print(f"Overall MSE for {offset} days ahead: {overall_mse:.4f}")
    print(f"Overall MAE for {offset} days ahead: {overall_mae:.4f}")


# CNN+Transformer
**Pediction All station**

In [ ]:
import os
import numpy as np
from datetime import datetime
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Reshape, Input, LayerNormalization, MultiHeadAttention, Dropout, Add, Concatenate, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

def load_tif_image(file_path, target_size=(128, 128)):
    img = load_img(file_path, color_mode='grayscale', target_size=target_size)
    img = img_to_array(img)
    img = img / 255.0  # Normalize to [0, 1]
    return img

def load_dataset_from_directory(directory, target_size=(128, 128)):
    images = []
    dates = []
    stations = []

    for file_name in sorted(os.listdir(directory)):
        if file_name.endswith('.tif'):
            try:
                station = file_name[:-15]
                date_str = file_name[-14:-4]
                date = datetime.strptime(date_str, '%Y-%m-%d')

                img_path = os.path.join(directory, file_name)
                print(f"Loading image: {img_path}")
                img = load_tif_image(img_path, target_size=target_size)
                images.append(img)
                dates.append(date)
                stations.append(station)
            except ValueError as e:
                print(f"Skipping file {file_name}: {e}")

    return np.array(images), np.array(dates), np.array(stations)

def prepare_data(images, sequence_length=10, prediction_offset=1):
    X = []
    y = []
    for i in range(len(images) - sequence_length - prediction_offset + 1):
        X.append(images[i:i + sequence_length])
        y.append(images[i + sequence_length + prediction_offset - 1])
    return np.array(X), np.array(y)

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = LayerNormalization(epsilon=1e-6)(inputs)
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = Dropout(dropout)(x)
    res = Add()([x, inputs])

    # Feed Forward Part
    x = LayerNormalization(epsilon=1e-6)(res)
    x = Dense(ff_dim, activation="relu")(x)
    x = Dropout(dropout)(x)
    x = Dense(inputs.shape[-1])(x)
    return Add()([x, res])

def create_cnn_transformer_model(sequence_length, image_shape, num_transformer_blocks=2, head_size=64, num_heads=2, ff_dim=64, dropout=0.1):
    model_input = Input(shape=(sequence_length, *image_shape))

    # CNN for feature extraction
    cnn_features = []
    for t in range(sequence_length):
        x = Conv2D(16, kernel_size=(3, 3), activation='relu')(model_input[:, t])
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Conv2D(32, kernel_size=(3, 3), activation='relu')(x)
        x = GlobalAveragePooling2D()(x)
        cnn_features.append(x)

    x = Concatenate()(cnn_features)
    x = Reshape((sequence_length, -1))(x)

    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = Flatten()(x)
    x_out = Dense(np.prod(image_shape), activation='sigmoid')(x)
    output = Reshape(image_shape)(x_out)

    model = Model(inputs=model_input, outputs=output)
    model.compile(optimizer='adam', loss='mean_squared_error')

    return model

def plot_image(img, title=''):
    plt.imshow(img, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

# Example usage
directory = '/content/drive/MyDrive/Dataset_img/one_month_NO2_2019'
stations = ['Delhi', 'Mumbai', 'Kolkata', 'Jaipur', 'Bengaluru', 'Bhopal', 'Chennai']
target_size = (128, 128)
sequence_length = 10
prediction_offsets = [1, 3, 5, 7]

all_images, all_dates, all_stations = load_dataset_from_directory(directory, target_size=target_size)

metrics = {'r2': [], 'mae': [], 'mse': [], 'rmse': []}

for prediction_offset in prediction_offsets:
    print(f"Training for {prediction_offset}-day ahead prediction")
    X, y = prepare_data(all_images, sequence_length=sequence_length, prediction_offset=prediction_offset)

    image_shape = (128, 128, 1)
    model = create_cnn_transformer_model(sequence_length, image_shape)

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model.fit(X, y, epochs=10, batch_size=4, validation_split=0.1, callbacks=[early_stopping])

    y_pred = model.predict(X)

    y_true_flat = y.reshape((len(y), -1))
    y_pred_flat = y_pred.reshape((len(y_pred), -1))

    r2 = r2_score(y_true_flat, y_pred_flat)
    mae = mean_absolute_error(y_true_flat, y_pred_flat)
    mse = mean_squared_error(y_true_flat, y_pred_flat)
    rmse = np.sqrt(mse)

    metrics['r2'].append(r2)
    metrics['mae'].append(mae)
    metrics['mse'].append(mse)
    metrics['rmse'].append(rmse)

    print(f"Metrics for {prediction_offset}-day ahead prediction:")
    print(f"R² score: {r2:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")

    # Plot an example prediction
    # plot_image(y[0].squeeze(), title=f'True Image - {prediction_offset} days ahead')
    # plot_image(y_pred[0].squeeze(), title=f'Predicted Image - {prediction_offset} days ahead')

average_r2 = np.mean(metrics['r2'])
average_mae = np.mean(metrics['mae'])
average_mse = np.mean(metrics['mse'])
average_rmse = np.mean(metrics['rmse'])

print("Overall Metrics across all prediction offsets:")
print(f"Overall R² score: {average_r2:.4f}")
print(f"Overall MAE: {average_mae:.4f}")
print(f"Overall MSE: {average_mse:.4f}")
print(f"Overall RMSE: {average_rmse:.4f}")


In [ ]:
import os
import numpy as np
from datetime import datetime
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Reshape, Input, Dropout, Add, Concatenate, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

def load_tif_image(file_path, target_size=(128, 128)):
    img = load_img(file_path, color_mode='grayscale', target_size=target_size)
    img = img_to_array(img)
    img = img / 255.0  # Normalize to [0, 1]
    return img

def load_dataset_from_directory(directory, target_size=(128, 128)):
    images = []
    dates = []
    stations = []

    for file_name in sorted(os.listdir(directory)):
        if file_name.endswith('.tif'):
            try:
                station = file_name[:-15]
                date_str = file_name[-14:-4]
                date = datetime.strptime(date_str, '%Y-%m-%d')

                img_path = os.path.join(directory, file_name)
                print(f"Loading image: {img_path}")
                img = load_tif_image(img_path, target_size=target_size)
                images.append(img)
                dates.append(date)
                stations.append(station)
            except ValueError as e:
                print(f"Skipping file {file_name}: {e}")

    return np.array(images), np.array(dates), np.array(stations)

def prepare_data(images, sequence_length=10, prediction_offset=1):
    X = []
    y = []
    for i in range(len(images) - sequence_length - prediction_offset + 1):
        X.append(images[i:i + sequence_length])
        y.append(images[i + sequence_length + prediction_offset - 1])
    return np.array(X), np.array(y)

def create_masknet_model(sequence_length, image_shape, dropout=0.1):
    model_input = Input(shape=(sequence_length, *image_shape))

    # CNN for feature extraction
    cnn_features = []
    for t in range(sequence_length):
        x = Conv2D(16, kernel_size=(3, 3), activation='relu')(model_input[:, t])
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Conv2D(32, kernel_size=(3, 3), activation='relu')(x)
        x = GlobalAveragePooling2D()(x)
        cnn_features.append(x)

    x = Concatenate()(cnn_features)
    x = Flatten()(x)

    # MaskNet fully connected layers
    x = Dense(256, activation='relu')(x)
    x = Dropout(dropout)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(dropout)(x)
    x_out = Dense(np.prod(image_shape), activation='sigmoid')(x)
    output = Reshape(image_shape)(x_out)

    model = Model(inputs=model_input, outputs=output)
    model.compile(optimizer='adam', loss='mean_squared_error')

    return model

def plot_image(img, title=''):
    plt.imshow(img, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

# Example usage
directory = '/content/drive/MyDrive/Dataset_img/one_month_NO2_2019'
target_size = (128, 128)
sequence_length = 10
prediction_offsets = [1, 3, 5, 7]

all_images, all_dates, all_stations = load_dataset_from_directory(directory, target_size=target_size)

metrics = {'r2': [], 'mae': [], 'mse': [], 'rmse': []}

for prediction_offset in prediction_offsets:
    print(f"Training for {prediction_offset}-day ahead prediction")
    X, y = prepare_data(all_images, sequence_length=sequence_length, prediction_offset=prediction_offset)

    image_shape = (128, 128, 1)
    model = create_masknet_model(sequence_length, image_shape)

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model.fit(X, y, epochs=10, batch_size=4, validation_split=0.1, callbacks=[early_stopping])

    y_pred = model.predict(X)

    y_true_flat = y.reshape((len(y), -1))
    y_pred_flat = y_pred.reshape((len(y_pred), -1))

    r2 = r2_score(y_true_flat, y_pred_flat)
    mae = mean_absolute_error(y_true_flat, y_pred_flat)
    mse = mean_squared_error(y_true_flat, y_pred_flat)
    rmse = np.sqrt(mse)

    metrics['r2'].append(r2)
    metrics['mae'].append(mae)
    metrics['mse'].append(mse)
    metrics['rmse'].append(rmse)

    print(f"Metrics for {prediction_offset}-day ahead prediction:")
    print(f"R² score: {r2:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")

    # Plot an example prediction
    # plot_image(y[0].squeeze(), title=f'True Image - {prediction_offset} days ahead')
    # plot_image(y_pred[0].squeeze(), title=f'Predicted Image - {prediction_offset} days ahead')

overall_r2 = np.mean(metrics['r2'])
overall_mae = np.mean(metrics['mae'])
overall_mse = np.mean(metrics['mse'])
overall_rmse = np.mean(metrics['rmse'])

print("Overall Metrics across all prediction offsets:")
print(f"Overall R² score: {overall_r2:.4f}")
print(f"Overall MAE: {overall_mae:.4f}")
print(f"Overall MSE: {overall_mse:.4f}")
print(f"Overall RMSE: {overall_rmse:.4f}")
